# BlueSky Flyer that uses the synApps busy record

## Python imports and definitions

Here are the full set of packages to imported.  The first block are Python standard packages, then come the ophyd and BluSky packages.  Just the parts we plan on using here.  Since this is also a tutorial, we will not rename imports or use other such shortcuts in the documentation (the online code has some shortcuts).

In [1]:
import enum
import logging
import threading
import time

import ophyd
import bluesky
import bluesky.plans

RE = bluesky.RunEngine({})
logger = logging.getLogger()

## EPICS PV names

These are the EPICS PV names we'll be using for the demo

In [2]:
BUSY_PV = 'prj:mybusy'
TIME_WAVE_PV = 'prj:t_array'
X_WAVE_PV = 'prj:x_array'
Y_WAVE_PV = 'prj:y_array'

## External pseudo-controller for fly scan

We'll need something that responds to the EPICS PV that signals a fly scan should be started.  It should operate the fly scan and store the collected data in EPICS PVs, then signal EPICS that the fly scan is complete.  BlueSky will observe that the fly scan is complete and collect the data from EPICS.

A Python program has been built to act as the pseudo-controller.  It step scans a soft [motor](https://github.com/epics-modules/motor) record and reads from an [swait](https://github.com/epics-modules/calc) record that has been configure to generate random numbers.

The [pseudo-controller](https://github.com/prjemian/ipython_mintvm/blob/master/profile_bluesky/startup/local_code/busyExample.py) *and associated EPICS IOC* should be started outside of this jupyter notebook.  Since the interface is completely through EPICS PVs, it is not necessary for either the IOC or the pseudo-controller to be running on the some computer as the jupyter notebook.


![control screen for the motor record](motor.png "control screen for the motor record")

![control screen for the swait record](swait.png "control screen for the swait record")

## Devices and support

Define some ophyd Devices we'll use and support routines for them.

### synApps busy record

The synApps [busy](https://github.com/epics-modules/busy) record is a variation of a 0,1 (bit).  It is an integer that starts at zero.  *Busy* is set to one at the start of operations.  This triggers execution of one or more operations that must complete before *busy* is set back to zero.  Each new operation increments *busy* at its start, then decrements *busy* when it is done.

![control screen for the busy record](busy.png "control screen for the busy record")

In [3]:
class BusyRecord(ophyd.Device):
    """a busy record sets the fly scan into action"""
    state = ophyd.Component(ophyd.EpicsSignal, "", string=True)
    output_link = ophyd.Component(ophyd.EpicsSignal, ".OUT")
    forward_link = ophyd.Component(ophyd.EpicsSignal, ".FLNK")

It is useful to define the state values (strings) so that clients have no ambiguity in spelling and error checking.

use this | instead of
---- | ----
`BusyStatus.busy` | `"Busy"`
`BusyStatus.done` | `"Done"`

In [4]:
class BusyStatus(str, enum.Enum):
    busy = "Busy"
    done = "Done"

We'll need a couple items from the [waveform](https://wiki-ext.aps.anl.gov/epics/index.php/RRM_3-14_Waveform) record.  Let's use a custom Device for just what we need here.

In [5]:
class MyWaveform(ophyd.Device):
    """waveform records store fly scan data"""
    wave = ophyd.Component(ophyd.EpicsSignalRO, "")
    number_elements = ophyd.Component(ophyd.EpicsSignalRO, ".NELM")
    number_read = ophyd.Component(ophyd.EpicsSignalRO, ".NORD")

In [6]:
## basic outline of a Flyer

#  (We are not yet using the pseudo-controller.  That comes later.)

In [7]:
class MyFlyer(ophyd.Device):
    """
    build a Flyer that we understand
    """

    xArr = ophyd.Component(MyWaveform, 'prj:x_array')
    # yArr = ophyd.Component(MyWaveform, 'prj:y_array')

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        return self._completion_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        st = ophyd.DeviceStatus(self)
        st._finished(success=True)
        return st

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        return {'ifly': {}
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")

In [8]:
ifly = MyFlyer(name="ifly")

In [9]:
# explain why to use these diagnostic steps routinely during development

# skip this next step now
# usually returns True, raises RuntimeError because we trap for "collection in progress"
#ifly.complete()

In [10]:
ifly.describe_collect()

{'ifly': {}}

In [11]:
ifly.collect()

We don't expect this to work correctly since it will not collect any data.  That's next.  That's why we don't try to run it, like this:

```
RE(bluesky.plans.fly([ifly]))
```

## First working Flyer - trivial data

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/062d1765023a4d9...388eb30304e51).

To collect data, we need to modify both the `collect()` *and* the `describe_collect()` methods.  BlueSky needs to know what kind of data to expect from this Flyer, so that it can generate the correct `descriptor` document.

For the *most* trivial case, we'll return a single number (`1.2345`) as the result of the first working Flyer.  (Still not yet using the pseudo-controller.)

In the `describe_collect()` method, we create a dictionary that describes the data to be collected:

        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

Then, in the `collect()` method, add the actual data collection code:

        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d


In [12]:
class MyFlyer(ophyd.Device):
    """
    build a Flyer that we understand
    """

    xArr = ophyd.Component(MyWaveform, 'prj:x_array')
    # yArr = ophyd.Component(MyWaveform, 'prj:y_array')

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        return self._completion_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        st = ophyd.DeviceStatus(self)
        st._finished(success=True)
        return st

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "fictional",
            dtype = "number",
            shape = []
        )
        return {
            'ifly': {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        t = time.time()
        d = dict(
            time=t,
            data=dict(x=1.2345),
            timestamps=dict(x=t)
        )
        yield d

As before, create a new instance of the *revised* `MyFlyer` class.

In [13]:
ifly = MyFlyer(name="ifly")

In [14]:
print('output from describe_collect() : ', ifly.describe_collect())
print("list output from collect() : ", list(ifly.collect()))

output from describe_collect() :  {'ifly': {'x': {'source': 'fictional', 'dtype': 'number', 'shape': []}}}
list output from collect() :  [{'time': 1524773091.672362, 'data': {'x': 1.2345}, 'timestamps': {'x': 1524773091.672362}}]


Running this flyer with the RunEngine seems anticlimactic in a jupyter notebook since we do not use a DataBroker, but the lack of exceptions tells us that it ran and we get a UUID at the end.

In [15]:
RE(bluesky.plans.fly([ifly]))

AttributeError: module 'bluesky' has no attribute 'plans'

## Flyer that "collects" array data

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/388eb30304e51...a0af3ec57a3430e777b3).

```
document that we generate 5 random numbers as an "array" for the `collect()` method.  Show what's been added.

explain the use of time.time and self.t0
```

In [ ]:
class MyFlyer(ophyd.Device):
    """
    build a Flyer that we understand
    """

    xArr = ophyd.Component(MyWaveform, 'prj:x_array')
    # yArr = ophyd.Component(MyWaveform, 'prj:y_array')

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None
        self.t0 = 0

    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # TODO: do the activity here
        # TODO: wait for completion
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        self.t0 = time.time()
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        return self._completion_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        st = ophyd.DeviceStatus(self)
        st._finished(success=True)
        return st

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        d = dict(
            source = "elapsed time, s",
            dtype = "number",
            shape = (1,)
        )
        return {
            'ifly': {
                "x": d
            }
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        for _ in range(5):
            t = time.time()
            x = t - self.t0 # data is elapsed time since kickoff()
            d = dict(
                time=t,
                data=dict(x=x),
                timestamps=dict(x=t)
            )
            yield d


In [ ]:
ifly = MyFlyer(name="ifly")
print('output from describe_collect() : ', ifly.describe_collect())
print("list output from collect() : ", list(ifly.collect()))

Again, not much information from running this flyer, except that it succeeds and a uuid is returned.

In [ ]:
RE(bluesky.plans.fly([ifly]))

## Final, working Flyer

See GitHub for a [summary of changes in source code](https://github.com/prjemian/ipython_mintvm/compare/a0af3ec57a3430e777b3...ce116e5e05774).

In [ ]:
# document what changed to get the final code, connected with the EPICS PVs

In [ ]:
class MyFlyer(ophyd.Device):
    """
    a basic Flyer for scans triggered by the synApps busy record
    """

    busy = ophyd.Component(BusyRecord, BUSY_PV)
    tArr = ophyd.Component(MyWaveform, TIME_WAVE_PV)
    xArr = ophyd.Component(MyWaveform, X_WAVE_PV)
    yArr = ophyd.Component(MyWaveform, Y_WAVE_PV)

    def __init__(self, *args, **kwargs):
        super().__init__('', parent=None, **kwargs)
        self._completion_status = None
        self.poll_sleep_interval_s = 0.05
        self.t0 = 0

    def wait_busy(self, target = None):
        """
        wait for the busy record to return to the target value
        """
        logger.debug("wait_busy()")
        target = target or BusyStatus.done

        while self.busy.state.value not in (target):
            time.sleep(self.poll_sleep_interval_s)  # wait to complete ...
 
    def my_activity(self):
        """
        start the "fly scan" here, could wait for completion
        
        It's OK to use blocking calls here 
        since this is called in a separate thread
        from the BlueSky RunEngine.
        """
        logger.info("activity()")
        if self._completion_status is None:
            logger.info("leaving activity() - not complete")
            return
        
        # do the activity here
        self.busy.state.put(BusyStatus.done) # make sure it's Done first
        self.wait_busy()

        # wait for completion
        self.t0 = time.time()
        self.busy.state.put(BusyStatus.busy)
        self.wait_busy()
        
        self._completion_status._finished(success=True)
        logger.info("activity() complete. status = " + str(self._completion_status))

    def kickoff(self):
        """
        Start this Flyer
        """
        logger.info("kickoff()")
        self._completion_status = ophyd.DeviceStatus(self)
        
        thread = threading.Thread(target=self.my_activity, daemon=True)
        thread.start()

        return self._completion_status

    def complete(self):
        """
        Wait for flying to be complete
        """
        logger.info("complete()")
        if self._completion_status is None:
            raise RuntimeError("No collection in progress")

        st = ophyd.DeviceStatus(self)
        st._finished(success=True)
        return st

    def describe_collect(self):
        """
        Describe details for ``collect()`` method
        """
        logger.info("describe_collect()")
        return {
            self.name: dict(
                ifly_xArr = dict(
                    source = self.xArr.wave.pvname,
                    dtype = "number",
                    shape = (1,)
                ),
                ifly_yArr = dict(
                    source = self.yArr.wave.pvname,
                    dtype = "number",
                    shape = (1,)
                ),
                ifly_tArr = dict(
                    source = self.tArr.wave.pvname,
                    dtype = "number",
                    shape = (1,)
                )
            )
        }

    def collect(self):
        """
        Start this Flyer
        """
        logger.info("collect()")
        for i in range(len(ifly.tArr.wave.value)):
            t = ifly.tArr.wave.value[i]
            x = ifly.xArr.wave.value[i]
            y = ifly.yArr.wave.value[i]
            d = dict(
                time=time.time(),
                data=dict(
                    ifly_tArr = time.time() - self.t0,
                    ifly_xArr = x,
                    ifly_yArr = y,
                ),
                timestamps=dict(
                    ifly_tArr = t,
                    ifly_xArr = t,
                    ifly_yArr = t,
                )
            )
            yield d


In [ ]:
ifly = MyFlyer("prj:", name="ifly")

Verify that we connected with the busy record, *et al.* by printing the current state.

In [ ]:
print(ifly.busy.state.pvname, ifly.busy.state.value)

In [ ]:
ifly.describe_collect()

In [ ]:
list(ifly.collect())

In [ ]:
RE(bluesky.plans.fly([ifly]), md=dict(purpose="develop Flyer for APS fly scans"))